In [1]:
import sqlite3
import pandas as pd
from fuzzywuzzy import process, fuzz
pd.options.display.max_rows = 2000
from sqlalchemy import create_engine

In [2]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("Resources/database.sqlite")
cur = con.cursor()

In [3]:
# Get selected fields from pitchfork reviews and genres tables
pitchfork_reviews_df = pd.read_sql_query("SELECT r.reviewid, r.title, r.artist, r.url, r.score, r.author, r.pub_date, g.genre \
from reviews r JOIN genres g WHERE r.reviewid = g.reviewid", con)
# Verify that result of SQL query is stored in the dataframe
#print(pitchfork_reviews_df.head())

In [4]:
# Be sure to close the connection
con.close()

In [5]:
#import the Rolling Stones Data CSV file
csv_file = ("Resources/albumlist.csv")
rolling_stones_df = pd.read_csv(csv_file,encoding= 'unicode_escape')
rolling_stones_df

,Number,Year,Album,Artist,Genre,Subgenre
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,Rock,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,Rock,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,Rock,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,Rock,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,"Rock, Pop",Pop Rock
5,6,1971,What's Going On,Marvin Gaye,Funk / Soul,Soul
6,7,1972,Exile on Main St.,The Rolling Stones,Rock,"Blues Rock, Rock & Roll, Classic Rock"
7,8,1979,London Calling,The Clash,Rock,"Punk, New Wave"
8,9,1966,Blonde on Blonde,Bob Dylan,"Rock, Blues","Folk Rock, Rhythm & Blues"
9,10,1968,"The Beatles (""The White Album"")",The Beatles,Rock,"Rock & Roll, Pop Rock, Psychedelic Rock, Exper..."


In [6]:
# Create a filtered dataframe from specific columns
rolling_stones_cols = ['Number', 'Year', 'Album', 'Artist', 'Subgenre']
rolling_stones_transformed = rolling_stones_df[rolling_stones_cols].copy()

# Rename the column headers
rolling_stones_transformed = rolling_stones_transformed.rename(columns={"Number": "id_rolling_stones_ranking",
                                                         "Year": "year_released",
                                                         "Album": "album_name",
                                                         "Subgenre": "subgenre", 
                                                        "Artist":"artist_name"})

# Clean the data by setting the index

rolling_stones_transformed = rolling_stones_transformed.reset_index()
rolling_stones_transformed = rolling_stones_transformed.rename(columns={"index": "id"})
rolling_stones_transformed.set_index("id", inplace=True)

# rolling_stones_transformed = rolling_stones_transformed.set_index("ID_rolling_stones_ranking")
rolling_stones_transformed
rolling_tosql_df = rolling_stones_transformed.copy()


In [7]:
rolling_tosql_df

,id_rolling_stones_ranking,year_released,album_name,artist_name,subgenre
id,,,,,
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock"
1,2,1966,Pet Sounds,The Beach Boys,"Pop Rock, Psychedelic Rock"
2,3,1966,Revolver,The Beatles,"Psychedelic Rock, Pop Rock"
3,4,1965,Highway 61 Revisited,Bob Dylan,"Folk Rock, Blues Rock"
4,5,1965,Rubber Soul,The Beatles,Pop Rock
5,6,1971,What's Going On,Marvin Gaye,Soul
6,7,1972,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock"
7,8,1979,London Calling,The Clash,"Punk, New Wave"
8,9,1966,Blonde on Blonde,Bob Dylan,"Folk Rock, Rhythm & Blues"


In [8]:
rolling_tosql_df.dtypes

id_rolling_stones_ranking     int64
year_released                 int64
album_name                   object
artist_name                  object
subgenre                     object
dtype: object

In [9]:
# Create a filtered dataframe from specific columns
pitchfork_cols = ['reviewid', 'artist', 'title','url', 'score', 'author', 'pub_date','genre']
pitchfork_reviews_transformed = pitchfork_reviews_df[pitchfork_cols].copy()

# Rename the column headers
pitchfork_reviews_transformed = pitchfork_reviews_transformed.rename(columns={"reviewid": "review_id",
                                                        "artist": "artist_name", 
                                                        "title": "album_name",
                                                         "score": "pitchfork_score"})

# Clean the data by dropping duplicates and setting the index
pitchfork_reviews_transformed.drop_duplicates("review_id", inplace=True)
pitchfork_reviews_transformed = pitchfork_reviews_transformed.reset_index()
pitchfork_reviews_transformed = pitchfork_reviews_transformed.rename(columns={"index": "id"})
pitchfork_reviews_transformed.set_index("id", inplace=True)
pitchfork_reviews_transformed.head(500)
pitchfork_tosql_df = pitchfork_reviews_transformed.copy()

In [10]:
pitchfork_tosql_df

,review_id,artist_name,album_name,url,pitchfork_score,author,pub_date,genre
id,,,,,,,,
0,22703,massive attack,mezzanine,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,nate patrin,2017-01-08,electronic
1,22721,krallice,prelapsarian,http://pitchfork.com/reviews/albums/22721-prel...,7.9,zoe camp,2017-01-07,metal
2,22659,uranium club,all of them naturals,http://pitchfork.com/reviews/albums/22659-all-...,7.3,david glickman,2017-01-07,rock
3,22661,"kleenex, liliput",first songs,http://pitchfork.com/reviews/albums/22661-firs...,9.0,jenn pelly,2017-01-06,rock
4,22725,taso,new start,http://pitchfork.com/reviews/albums/22725-new-...,8.1,kevin lozano,2017-01-06,electronic
...,...,...,...,...,...,...,...,...
22682,1535,coldcut,let us replay!,http://pitchfork.com/reviews/albums/1535-let-u...,8.9,james p. wisdom,1999-01-26,electronic
22684,1341,cassius,1999,http://pitchfork.com/reviews/albums/1341-1999/,4.8,james p. wisdom,1999-01-26,electronic
22685,5376,mojave 3,out of tune,http://pitchfork.com/reviews/albums/5376-out-o...,6.3,jason josephes,1999-01-12,rock


In [11]:
rolling_stones_transformed.columns

Index(['id_rolling_stones_ranking', 'year_released', 'album_name',
       'artist_name', 'subgenre'],
      dtype='object')

In [12]:
# lowercase the album name and artist name in the rolling stone list to match with pitchfork list
album = [str(i).lower() for i in rolling_stones_transformed['album_name']]
artist = [str(i).lower() for i in rolling_stones_transformed['artist_name']]
rolling_stones_transformed['artist'] = artist
rolling_stones_transformed['album'] = album
rolling_stones_transformed

,id_rolling_stones_ranking,year_released,album_name,artist_name,subgenre,artist,album
id,,,,,,,
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",the beatles,sgt. pepper's lonely hearts club band
1,2,1966,Pet Sounds,The Beach Boys,"Pop Rock, Psychedelic Rock",the beach boys,pet sounds
2,3,1966,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",the beatles,revolver
3,4,1965,Highway 61 Revisited,Bob Dylan,"Folk Rock, Blues Rock",bob dylan,highway 61 revisited
4,5,1965,Rubber Soul,The Beatles,Pop Rock,the beatles,rubber soul
5,6,1971,What's Going On,Marvin Gaye,Soul,marvin gaye,what's going on
6,7,1972,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",the rolling stones,exile on main st.
7,8,1979,London Calling,The Clash,"Punk, New Wave",the clash,london calling
8,9,1966,Blonde on Blonde,Bob Dylan,"Folk Rock, Rhythm & Blues",bob dylan,blonde on blonde


In [13]:
# In Rolling Stones review list, concatenate the lowercased 'artist' and 'album' strings into a new column 'concat'.
rolling_stones_transformed['concat'] = rolling_stones_transformed['artist'] + rolling_stones_transformed['album'] 
rolling_stones_transformed['concat']
rolling_stones_transformed['alpha_concat'] = rolling_stones_transformed['concat'].map

yo = "the beatlessgt. pepper's lonely hearts club band"

# Remove non alphanumeric characters from the concatenated strings

def my_filter(my_str):
    a_string = my_str
    alphanumeric_filter = filter(str.isalnum, a_string)
    alphanumeric_string = "".join(alphanumeric_filter)
    return alphanumeric_string

rolling_stones_transformed['alpha_concat'] = rolling_stones_transformed['concat'].apply(my_filter)
rolling_stones_transformed


,id_rolling_stones_ranking,year_released,album_name,artist_name,subgenre,artist,album,concat,alpha_concat
id,,,,,,,,,
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",the beatles,sgt. pepper's lonely hearts club band,the beatlessgt. pepper's lonely hearts club band,thebeatlessgtpepperslonelyheartsclubband
1,2,1966,Pet Sounds,The Beach Boys,"Pop Rock, Psychedelic Rock",the beach boys,pet sounds,the beach boyspet sounds,thebeachboyspetsounds
2,3,1966,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",the beatles,revolver,the beatlesrevolver,thebeatlesrevolver
3,4,1965,Highway 61 Revisited,Bob Dylan,"Folk Rock, Blues Rock",bob dylan,highway 61 revisited,bob dylanhighway 61 revisited,bobdylanhighway61revisited
4,5,1965,Rubber Soul,The Beatles,Pop Rock,the beatles,rubber soul,the beatlesrubber soul,thebeatlesrubbersoul
5,6,1971,What's Going On,Marvin Gaye,Soul,marvin gaye,what's going on,marvin gayewhat's going on,marvingayewhatsgoingon
6,7,1972,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",the rolling stones,exile on main st.,the rolling stonesexile on main st.,therollingstonesexileonmainst
7,8,1979,London Calling,The Clash,"Punk, New Wave",the clash,london calling,the clashlondon calling,theclashlondoncalling
8,9,1966,Blonde on Blonde,Bob Dylan,"Folk Rock, Rhythm & Blues",bob dylan,blonde on blonde,bob dylanblonde on blonde,bobdylanblondeonblonde


In [14]:
# In Pitchfork review list, concatenate the 'artist_name' and 'album_name' strings into a new column 'concat'
pitchfork_reviews_transformed['concat'] = pitchfork_reviews_transformed['artist_name'] + pitchfork_reviews_transformed['album_name']

# Revmove non alphanumeric characters from concatenated pitchfork artist / album string. 
pitchfork_reviews_transformed['alpha_concat'] = pitchfork_reviews_transformed['concat'].apply(my_filter)

pitchfork_reviews_transformed


,review_id,artist_name,album_name,url,pitchfork_score,author,pub_date,genre,concat,alpha_concat
id,,,,,,,,,,
0,22703,massive attack,mezzanine,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,nate patrin,2017-01-08,electronic,massive attackmezzanine,massiveattackmezzanine
1,22721,krallice,prelapsarian,http://pitchfork.com/reviews/albums/22721-prel...,7.9,zoe camp,2017-01-07,metal,kralliceprelapsarian,kralliceprelapsarian
2,22659,uranium club,all of them naturals,http://pitchfork.com/reviews/albums/22659-all-...,7.3,david glickman,2017-01-07,rock,uranium cluball of them naturals,uraniumcluballofthemnaturals
3,22661,"kleenex, liliput",first songs,http://pitchfork.com/reviews/albums/22661-firs...,9.0,jenn pelly,2017-01-06,rock,"kleenex, liliputfirst songs",kleenexliliputfirstsongs
4,22725,taso,new start,http://pitchfork.com/reviews/albums/22725-new-...,8.1,kevin lozano,2017-01-06,electronic,tasonew start,tasonewstart
...,...,...,...,...,...,...,...,...,...,...
22682,1535,coldcut,let us replay!,http://pitchfork.com/reviews/albums/1535-let-u...,8.9,james p. wisdom,1999-01-26,electronic,coldcutlet us replay!,coldcutletusreplay
22684,1341,cassius,1999,http://pitchfork.com/reviews/albums/1341-1999/,4.8,james p. wisdom,1999-01-26,electronic,cassius1999,cassius1999
22685,5376,mojave 3,out of tune,http://pitchfork.com/reviews/albums/5376-out-o...,6.3,jason josephes,1999-01-12,rock,mojave 3out of tune,mojave3outoftune


In [15]:
# fuzzywuzzy (i'll be adding some more comments here)
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # return empty string if no match exists
    max_name = ""
    # Iterate over over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [16]:
dict_list = []
# iterate over rolling_stones_transformed
for name in rolling_stones_transformed['alpha_concat']:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, pitchfork_reviews_transformed.alpha_concat, 80)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"player_name" : name})
    dict_.update({"match_name" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
fuzzy_table = pd.DataFrame(dict_list)
fuzzy_table

,player_name,match_name,score
0,thebeatlessgtpepperslonelyheartsclubband,thebeatlessgtpepperslonelyheartsclubband,100
1,thebeachboyspetsounds,,-1
2,thebeatlesrevolver,thebeatlesrevolver,100
3,bobdylanhighway61revisited,,-1
4,thebeatlesrubbersoul,thebeatlesrubbersoul,100
5,marvingayewhatsgoingon,,-1
6,therollingstonesexileonmainst,therollingstonesexileonmainstdeluxeedition,82
7,theclashlondoncalling,,-1
8,bobdylanblondeonblonde,,-1
9,thebeatlesthebeatlesthewhitealbum,,-1


In [17]:
rolling_stones_transformed['match_name'] = fuzzy_table['match_name']
rolling_stones_transformed['match_score'] = fuzzy_table['score']
rolling_stones_transformed

,id_rolling_stones_ranking,year_released,album_name,artist_name,subgenre,artist,album,concat,alpha_concat,match_name,match_score
id,,,,,,,,,,,
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",the beatles,sgt. pepper's lonely hearts club band,the beatlessgt. pepper's lonely hearts club band,thebeatlessgtpepperslonelyheartsclubband,thebeatlessgtpepperslonelyheartsclubband,100
1,2,1966,Pet Sounds,The Beach Boys,"Pop Rock, Psychedelic Rock",the beach boys,pet sounds,the beach boyspet sounds,thebeachboyspetsounds,,-1
2,3,1966,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",the beatles,revolver,the beatlesrevolver,thebeatlesrevolver,thebeatlesrevolver,100
3,4,1965,Highway 61 Revisited,Bob Dylan,"Folk Rock, Blues Rock",bob dylan,highway 61 revisited,bob dylanhighway 61 revisited,bobdylanhighway61revisited,,-1
4,5,1965,Rubber Soul,The Beatles,Pop Rock,the beatles,rubber soul,the beatlesrubber soul,thebeatlesrubbersoul,thebeatlesrubbersoul,100
5,6,1971,What's Going On,Marvin Gaye,Soul,marvin gaye,what's going on,marvin gayewhat's going on,marvingayewhatsgoingon,,-1
6,7,1972,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",the rolling stones,exile on main st.,the rolling stonesexile on main st.,therollingstonesexileonmainst,therollingstonesexileonmainstdeluxeedition,82
7,8,1979,London Calling,The Clash,"Punk, New Wave",the clash,london calling,the clashlondon calling,theclashlondoncalling,,-1
8,9,1966,Blonde on Blonde,Bob Dylan,"Folk Rock, Rhythm & Blues",bob dylan,blonde on blonde,bob dylanblonde on blonde,bobdylanblondeonblonde,,-1


In [18]:
# Drop unmatched rows
rolling_drop_unmatched_df = rolling_stones_transformed.drop(rolling_stones_transformed[(rolling_stones_transformed['match_name'] == "")].index)
rolling_drop_unmatched_df


,id_rolling_stones_ranking,year_released,album_name,artist_name,subgenre,artist,album,concat,alpha_concat,match_name,match_score
id,,,,,,,,,,,
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",the beatles,sgt. pepper's lonely hearts club band,the beatlessgt. pepper's lonely hearts club band,thebeatlessgtpepperslonelyheartsclubband,thebeatlessgtpepperslonelyheartsclubband,100
2,3,1966,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",the beatles,revolver,the beatlesrevolver,thebeatlesrevolver,thebeatlesrevolver,100
4,5,1965,Rubber Soul,The Beatles,Pop Rock,the beatles,rubber soul,the beatlesrubber soul,thebeatlesrubbersoul,thebeatlesrubbersoul,100
6,7,1972,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",the rolling stones,exile on main st.,the rolling stonesexile on main st.,therollingstonesexileonmainst,therollingstonesexileonmainstdeluxeedition,82
13,14,1969,Abbey Road,The Beatles,"Psychedelic Rock, Classic Rock, Pop Rock",the beatles,abbey road,the beatlesabbey road,thebeatlesabbeyroad,thebeatlesabbeyroad,100
15,16,1975,Blood on the Tracks,Bob Dylan,"Folk Rock, Acoustic, Ballad",bob dylan,blood on the tracks,bob dylanblood on the tracks,bobdylanbloodonthetracks,bobdylanbloodonthetracks,100
18,19,1968,Astral Weeks,Van Morrison,"Acoustic, Classic Rock, Free Improvisation",van morrison,astral weeks,van morrisonastral weeks,vanmorrisonastralweeks,vanmorrisonastralweeks,100
25,26,1977,Rumours,Fleetwood Mac,Pop Rock,fleetwood mac,rumours,fleetwood macrumours,fleetwoodmacrumours,fleetwoodmacrumours,100
28,29,1969,Led Zeppelin,Led Zeppelin,"Blues Rock, Hard Rock",led zeppelin,led zeppelin,led zeppelinled zeppelin,ledzeppelinledzeppelin,ledzeppelinledzeppelin,100


In [19]:
#Drop the columns with errors
rolling_dropped_df = rolling_drop_unmatched_df.drop(rolling_drop_unmatched_df[
    (rolling_drop_unmatched_df['alpha_concat'] == "thebeatlesmeetthebeatles") |
    (rolling_drop_unmatched_df['alpha_concat'] == "ledzeppelinledzeppelinii") |
    (rolling_drop_unmatched_df['alpha_concat'] == "thevelvetundergroundloaded") |       
    (rolling_drop_unmatched_df['alpha_concat'] == "radioheadthebends") | 
    (rolling_drop_unmatched_df['alpha_concat'] == "iggyandthestoogesrawpower") |   
    (rolling_drop_unmatched_df['alpha_concat'] == "michaeljacksonbad") |   
    (rolling_drop_unmatched_df['alpha_concat'] == "thereplacementsletitbe") |   
    (rolling_drop_unmatched_df['alpha_concat'] == "talkingheadstalkingheads77") |   
    (rolling_drop_unmatched_df['alpha_concat'] == "leonardcohensongsofloveandhate") |   
    (rolling_drop_unmatched_df['alpha_concat'] == "thesmashingpumpkinssiamesedream")    
    ].index)
rolling_dropped_df


,id_rolling_stones_ranking,year_released,album_name,artist_name,subgenre,artist,album,concat,alpha_concat,match_name,match_score
id,,,,,,,,,,,
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",the beatles,sgt. pepper's lonely hearts club band,the beatlessgt. pepper's lonely hearts club band,thebeatlessgtpepperslonelyheartsclubband,thebeatlessgtpepperslonelyheartsclubband,100
2,3,1966,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",the beatles,revolver,the beatlesrevolver,thebeatlesrevolver,thebeatlesrevolver,100
4,5,1965,Rubber Soul,The Beatles,Pop Rock,the beatles,rubber soul,the beatlesrubber soul,thebeatlesrubbersoul,thebeatlesrubbersoul,100
6,7,1972,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",the rolling stones,exile on main st.,the rolling stonesexile on main st.,therollingstonesexileonmainst,therollingstonesexileonmainstdeluxeedition,82
13,14,1969,Abbey Road,The Beatles,"Psychedelic Rock, Classic Rock, Pop Rock",the beatles,abbey road,the beatlesabbey road,thebeatlesabbeyroad,thebeatlesabbeyroad,100
15,16,1975,Blood on the Tracks,Bob Dylan,"Folk Rock, Acoustic, Ballad",bob dylan,blood on the tracks,bob dylanblood on the tracks,bobdylanbloodonthetracks,bobdylanbloodonthetracks,100
18,19,1968,Astral Weeks,Van Morrison,"Acoustic, Classic Rock, Free Improvisation",van morrison,astral weeks,van morrisonastral weeks,vanmorrisonastralweeks,vanmorrisonastralweeks,100
25,26,1977,Rumours,Fleetwood Mac,Pop Rock,fleetwood mac,rumours,fleetwood macrumours,fleetwoodmacrumours,fleetwoodmacrumours,100
28,29,1969,Led Zeppelin,Led Zeppelin,"Blues Rock, Hard Rock",led zeppelin,led zeppelin,led zeppelinled zeppelin,ledzeppelinledzeppelin,ledzeppelinledzeppelin,100


In [20]:
rolling_dropped_df.count()

id_rolling_stones_ranking    111
year_released                111
album_name                   111
artist_name                  111
subgenre                     111
artist                       111
album                        111
concat                       111
alpha_concat                 111
match_name                   111
match_score                  111
dtype: int64

In [21]:
#rename the pitchfork alpha_concat column as match_name, to match the column name in other table for the join
pitchfork_transformed_renamed = pitchfork_reviews_transformed.rename(columns={
     "alpha_concat": "match_name"})
pitchfork_transformed_renamed

,review_id,artist_name,album_name,url,pitchfork_score,author,pub_date,genre,concat,match_name
id,,,,,,,,,,
0,22703,massive attack,mezzanine,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,nate patrin,2017-01-08,electronic,massive attackmezzanine,massiveattackmezzanine
1,22721,krallice,prelapsarian,http://pitchfork.com/reviews/albums/22721-prel...,7.9,zoe camp,2017-01-07,metal,kralliceprelapsarian,kralliceprelapsarian
2,22659,uranium club,all of them naturals,http://pitchfork.com/reviews/albums/22659-all-...,7.3,david glickman,2017-01-07,rock,uranium cluball of them naturals,uraniumcluballofthemnaturals
3,22661,"kleenex, liliput",first songs,http://pitchfork.com/reviews/albums/22661-firs...,9.0,jenn pelly,2017-01-06,rock,"kleenex, liliputfirst songs",kleenexliliputfirstsongs
4,22725,taso,new start,http://pitchfork.com/reviews/albums/22725-new-...,8.1,kevin lozano,2017-01-06,electronic,tasonew start,tasonewstart
...,...,...,...,...,...,...,...,...,...,...
22682,1535,coldcut,let us replay!,http://pitchfork.com/reviews/albums/1535-let-u...,8.9,james p. wisdom,1999-01-26,electronic,coldcutlet us replay!,coldcutletusreplay
22684,1341,cassius,1999,http://pitchfork.com/reviews/albums/1341-1999/,4.8,james p. wisdom,1999-01-26,electronic,cassius1999,cassius1999
22685,5376,mojave 3,out of tune,http://pitchfork.com/reviews/albums/5376-out-o...,6.3,jason josephes,1999-01-12,rock,mojave 3out of tune,mojave3outoftune


In [22]:
#merge the rolling stone table and the pitchfork table
merge_table = pd.merge(rolling_dropped_df, pitchfork_transformed_renamed, on="match_name", how="left")
merge_table


,id_rolling_stones_ranking,year_released,album_name_x,artist_name_x,subgenre,artist,album,concat_x,alpha_concat,match_name,match_score,review_id,artist_name_y,album_name_y,url,pitchfork_score,author,pub_date,genre,concat_y
0,1,1967,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",the beatles,sgt. pepper's lonely hearts club band,the beatlessgt. pepper's lonely hearts club band,thebeatlessgtpepperslonelyheartsclubband,thebeatlessgtpepperslonelyheartsclubband,100,13435,the beatles,sgt. pepper's lonely hearts club band,http://pitchfork.com/reviews/albums/13435-sgt-...,10.0,scott plagenhoef,2009-09-09,rock,the beatlessgt. pepper's lonely hearts club band
1,3,1966,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",the beatles,revolver,the beatlesrevolver,thebeatlesrevolver,thebeatlesrevolver,100,13434,the beatles,revolver,http://pitchfork.com/reviews/albums/13434-revo...,10.0,scott plagenhoef,2009-09-09,rock,the beatlesrevolver
2,5,1965,Rubber Soul,The Beatles,Pop Rock,the beatles,rubber soul,the beatlesrubber soul,thebeatlesrubbersoul,thebeatlesrubbersoul,100,13433,the beatles,rubber soul,http://pitchfork.com/reviews/albums/13433-rubb...,10.0,scott plagenhoef,2009-09-09,rock,the beatlesrubber soul
3,7,1972,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",the rolling stones,exile on main st.,the rolling stonesexile on main st.,therollingstonesexileonmainst,therollingstonesexileonmainstdeluxeedition,82,14264,the rolling stones,exile on main st. [deluxe edition],http://pitchfork.com/reviews/albums/14264-exil...,10.0,rob mitchum,2010-05-19,rock,the rolling stonesexile on main st. [deluxe ed...
4,14,1969,Abbey Road,The Beatles,"Psychedelic Rock, Classic Rock, Pop Rock",the beatles,abbey road,the beatlesabbey road,thebeatlesabbeyroad,thebeatlesabbeyroad,100,13431,the beatles,abbey road,http://pitchfork.com/reviews/albums/13431-abbe...,10.0,mark richardson,2009-09-10,rock,the beatlesabbey road
5,16,1975,Blood on the Tracks,Bob Dylan,"Folk Rock, Acoustic, Ballad",bob dylan,blood on the tracks,bob dylanblood on the tracks,bobdylanbloodonthetracks,bobdylanbloodonthetracks,100,22485,bob dylan,blood on the tracks,http://pitchfork.com/reviews/albums/22485-bloo...,10.0,jesse jarnow,2016-10-30,rock,bob dylanblood on the tracks
6,19,1968,Astral Weeks,Van Morrison,"Acoustic, Classic Rock, Free Improvisation",van morrison,astral weeks,van morrisonastral weeks,vanmorrisonastralweeks,vanmorrisonastralweeks,100,21092,van morrison,astral weeks,http://pitchfork.com/reviews/albums/21092-astr...,10.0,stephen thomas erlewine,2015-11-06,rock,van morrisonastral weeks
7,26,1977,Rumours,Fleetwood Mac,Pop Rock,fleetwood mac,rumours,fleetwood macrumours,fleetwoodmacrumours,fleetwoodmacrumours,100,17499,fleetwood mac,rumours,http://pitchfork.com/reviews/albums/17499-rumo...,10.0,jessica hopper,2013-02-08,rock,fleetwood macrumours
8,29,1969,Led Zeppelin,Led Zeppelin,"Blues Rock, Hard Rock",led zeppelin,led zeppelin,led zeppelinled zeppelin,ledzeppelinledzeppelin,ledzeppelinledzeppelin,100,19418,led zeppelin,led zeppelin,http://pitchfork.com/reviews/albums/19418-led-...,9.2,mark richardson,2014-06-12,rock,led zeppelinled zeppelin
9,39,1963,Please Please Me,The Beatles,"Beat, Rock & Roll",the beatles,please please me,the beatlesplease please me,thebeatlespleasepleaseme,thebeatlespleasepleaseme,100,13419,the beatles,please please me,http://pitchfork.com/reviews/albums/13419-plea...,9.5,tom ewing,2009-09-08,rock,the beatlesplease please me


In [24]:
# Extract the final columns that will be in our final table

merge_table_cols = ['album_name_x', 'artist_name_x', 'subgenre', 'id_rolling_stones_ranking', 'pitchfork_score']
merge_table_transformed = merge_table[merge_table_cols].copy()
merge_table_transformed


,album_name_x,artist_name_x,subgenre,id_rolling_stones_ranking,pitchfork_score
0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",1,10.0
1,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",3,10.0
2,Rubber Soul,The Beatles,Pop Rock,5,10.0
3,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",7,10.0
4,Abbey Road,The Beatles,"Psychedelic Rock, Classic Rock, Pop Rock",14,10.0
5,Blood on the Tracks,Bob Dylan,"Folk Rock, Acoustic, Ballad",16,10.0
6,Astral Weeks,Van Morrison,"Acoustic, Classic Rock, Free Improvisation",19,10.0
7,Rumours,Fleetwood Mac,Pop Rock,26,10.0
8,Led Zeppelin,Led Zeppelin,"Blues Rock, Hard Rock",29,9.2
9,Please Please Me,The Beatles,"Beat, Rock & Roll",39,9.5


In [27]:
# Rename the columns in our final table
merge_table_renamed = merge_table_transformed.rename(columns={"album_name_x": "album_name",
                                                         "artist_name_x": "artist_name"})

merge_table_renamed = merge_table_renamed.reset_index()
merge_table_renamed = merge_table_renamed.rename(columns={"index": "id"})
merge_table_renamed.set_index("id", inplace=True) 
merge_table_renamed

,album_name,artist_name,subgenre,id_rolling_stones_ranking,pitchfork_score
id,,,,,
0,Sgt. Pepper's Lonely Hearts Club Band,The Beatles,"Rock & Roll, Psychedelic Rock",1,10.0
1,Revolver,The Beatles,"Psychedelic Rock, Pop Rock",3,10.0
2,Rubber Soul,The Beatles,Pop Rock,5,10.0
3,Exile on Main St.,The Rolling Stones,"Blues Rock, Rock & Roll, Classic Rock",7,10.0
4,Abbey Road,The Beatles,"Psychedelic Rock, Classic Rock, Pop Rock",14,10.0
5,Blood on the Tracks,Bob Dylan,"Folk Rock, Acoustic, Ballad",16,10.0
6,Astral Weeks,Van Morrison,"Acoustic, Classic Rock, Free Improvisation",19,10.0
7,Rumours,Fleetwood Mac,Pop Rock,26,10.0
8,Led Zeppelin,Led Zeppelin,"Blues Rock, Hard Rock",29,9.2


In [35]:
connection_string = "postgres:postgres@localhost:5432/etl_music_db"
engine = create_engine(f'postgresql://{connection_string}')

In [36]:
engine.table_names()

['merged_table', 'pitchfork_reviews', 'rolling_stone']

In [37]:
#Add Rolling Stones dataframe to SQL Database
rolling_tosql_df.to_sql(name='rolling_stone', con=engine, if_exists='append', index=True)

In [38]:
#Add Pitchfork dataframe to SQL Database
pitchfork_tosql_df.to_sql(name='pitchfork_reviews', con=engine, if_exists='append', index=True)

In [39]:
#Add the merged_table_renamed dataframe to SQL Database
merge_table_renamed.to_sql(name='merged_table', con=engine, if_exists='append', index=True)